In [21]:
# bug incompatibilité dans myenv_clone, ok dans nlp_env
# faire du nettoyage dans les envs un jour …
import os
from ollama import Client

OLLAMA_HOST = os.environ.get("OLLAMA_HOST")
client = Client(host=OLLAMA_HOST)


In [22]:
import pandas as pd

df = pd.read_csv("../data/df_match.csv")

In [3]:
# # test
# response = client.chat(
#     model="llama3.2",
#     messages=[
#         {
#             "role": "user",
#             "content": "Why is the sky blue?",
#         },
#     ],
# )
# print(response)

model='llama3.2' created_at='2025-06-26T14:35:44.364221377Z' done=True done_reason='stop' total_duration=1291411302 load_duration=14430832 prompt_eval_count=31 prompt_eval_duration=1834207 eval_count=283 eval_duration=1274251929 message=Message(role='assistant', content="The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described it in the late 19th century.\n\nHere's what happens:\n\n1. Sunlight enters Earth's atmosphere and is made up of different wavelengths (colors) of light.\n2. The shorter (blue) wavelengths are scattered more than the longer (red) wavelengths by the tiny molecules of gases such as nitrogen and oxygen in the atmosphere.\n3. This scattering effect is more pronounced for blue light because it has a smaller wavelength, which means it's more easily deflected by the tiny molecules.\n4. As a result, when sunlight enters our eyes, we see mostly the scattered blue light, making the sky appe

In [39]:
# TODO: need to check prompt struct to be sure of what is going to ollama
def get_prompt(text):
    return [
        {
            "role": "system",
            "content": (
                "Tu es un expert en sciences politique. "
                "Classifie chaque intervention comme relevant du thème 'REPUBLIQUE' ou 'AUTRE'"
                "Réponds uniquement par un de ces deux mots"
            ),
        },
        {"role": "user", "content": f'Classe cette intervention :\n"{text}"\nLabel:'},
    ]


In [25]:
from tqdm import tqdm


def do_predictions(prompt_generator, texts, model):
    """
    Prediction avec Ollama : prompt_generator, liste de textes, modèle.
    Affiche la progression (tqdm).
    """
    results = []
    total = len(texts)

    with tqdm(
        total=total,
        desc="Progress",
        unit="item",
        bar_format="{l_bar}{bar} | {n_fmt}/{total_fmt} [{percentage:3.0f}%]",
    ) as pbar:
        for i, text in texts.items():
            try:
                messages = prompt_generator(text)
                response = client.chat(model=model, messages=messages)
                content = response.get("message", {}).get("content", "").strip()
                results.append(content)
            except Exception as e:
                print(f"Erreur ligne {i} : {e}")
                results.append(None)
            pbar.update(1)

    return results


In [ ]:
# # S'inspirer de ça si besoin ?
# # The structure is a bit different
# def build_prompt(text):
#     system_prompt = (
#         "You are a helpful and accurate news headline classifier. "
#         "Your job is to classify news headlines as either 'POLITICS' or 'OTHER'. "
#         "Only respond with exactly one of those two labels."
#     )
#     user_prompt = f"Classify this headline:\n\"{text}\"\nLabel:"
#     return f"<<SYS>>\n{system_prompt}\n<</SYS>>\n\n[INST] {user_prompt} [/INST]"

In [ ]:
import time
# si abesoin ajout pause, un truc du genre ?

# # test avec un extrait
# texts = df["Texte"][:100]  # ou .iloc[:blabla]

start = time.time()
# Appel
r_ollama = do_predictions(get_prompt, df["Texte"], model="llama3.1:70b")
end = time.time()
print(f"Temps d'exécution : {end - start:.2f} secondes")

# # Ajout dans le DataFrame
# df.loc[texts.index, "prediction_simple"] = r_ollama

# guess it kinda work ?
df.loc[df["Texte"].index, "prediction_simple"] = r_ollama


In [54]:
df_full_pred = df[["ID_paragraphe", "prediction_simple"]]

In [55]:
df_full_pred.to_csv("../data/intermediate/df_full_pred_simple.csv", index=False)

## longer prompt ?

In [ ]:
# TODO: need to check prompt struct to be sure of what is going to ollama


def get_prompt_long(text):
    return [
        {
            "role": "system",
            "content": (
                "Dans le cadre d’un article académique en science politique, tu dois distinguer les interventions relatives à la 'République' des 'autres' "
                "Il s’agit d’un corpus d’interventions de députés à l’assemblée nationale en France lors de la 16e législature."
                " La « République » renvoie ici à sa dimension idéelle ou idéologique. Est donc exclut les références à des pays, au nom d’un parti politique, d’un journaliste ou d’une institution, ou à la fonction de président de la République. Est inclut en plus du champ lexical de la « République », à la devise « liberté, égalité, fraternité » et aux principes de « laïcité », « universalisme » et « indivisibilité ». "
                "Classifie chaque intervention comme 'REPUBLIQUE' ou 'AUTRE'"
                "Réponds uniquement par un de ces deux mots."
            ),
        },
        {"role": "user", "content": f'Classe cette intervention :\n"{text}"\nLabel:'},
    ]


In [26]:
import time

# test avec un extrait
texts = df["Texte"][:100]  # ou .iloc[:10]

start = time.time()
# Appel
r_ollama = do_predictions(get_prompt_long, texts, model="llama3.1:70b")
end = time.time()
print(f"Temps d'exécution : {end - start:.2f} secondes")

# Ajout dans le DataFrame
df.loc[texts.index, "prediction"] = r_ollama

Progress: 100%|██████████ | 100/100 [100%]

Temps d'exécution : 78.47 secondes


# Experimentation

## Scores

In [58]:
from sklearn.metrics import classification_report, f1_score
df_score = pd.read_csv("../data/intermediate/df_merged_pred.csv")
df_score.head()

,ID_paragraphe,prediction_long,prediction
0,3043935,AUTRE,REPUBLIQUE
1,3044004,AUTRE,REPUBLIQUE
2,3044275,AUTRE,AUTRE
3,3044207,AUTRE,REPUBLIQUE
4,3044651,AUTRE,REPUBLIQUE


In [ ]:
# pour le principe, puisque ici pas de gold standard, juste les modèles entre eux
df_score = df_score[0:100]
print(classification_report(df_score["prediction_long"], df_score["prediction"], digits=3))

              precision    recall  f1-score   support

       AUTRE      0.625     0.172     0.270        29
      AUTRE.      0.000     0.000     0.000         0
  REPUBLIQUE      0.739     0.915     0.818        71
  République      0.000     0.000     0.000         0
 République.      0.000     0.000     0.000         0

    accuracy                          0.700       100
   macro avg      0.273     0.218     0.218       100
weighted avg      0.706     0.700     0.659       100



/Users/leo/anaconda3/envs/nlp_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/leo/anaconda3/envs/nlp_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/leo/anaconda3/envs/nlp_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [63]:
f1_score(df_score["prediction_long"], df_score["prediction"], average="macro")

0.21757606663267043

In [ ]:
# comparer les modèles
pd.crosstab(df_score["prediction_long"], df_score["prediction"])

prediction,AUTRE,AUTRE.,REPUBLIQUE,République,République.
prediction_long,,,,,
AUTRE,5,1,23,0,0
REPUBLIQUE,3,1,65,1,1


## Few-shot ?

In [ ]:
# # Let's build the prompt, with some examples
# few_shot_examples = [
#         ("Biden signs executive order on student debt relief", "POLITICS"),
#         ("Amazon launches new AI-powered Alexa features", "OTHER"),
#         ("Congress debates military spending bill", "POLITICS"),
#         ("PSG wins much-anticipated Champions League", "OTHER")
#         ]

# def get_prompt_few_shots(text: str, examples: list[tuple] = few_shot_examples):
#   examples = "\n".join([f"Classify this headline:\n{headline}\nLabel: {label}\n\n" for headline, label in examples])
#   return [{"role":"system",
#            "content":"You are a strict news classifier. You must respond with one word only — either 'POLITICS' or 'OTHER'." +
#            "\n Do not explain. Do not output anything else."

#            },
#            {"role":"user","content": f"{examples}\nClassify this headline:\n\"{text}\"\nLabel:"}
#   ]

## Refacto

In [ ]:
# # Inspiration GPT pour avoir des checkpoints :
# results = []
# indexes = []
# batch_size = 20

# for i in range(0, len(df), batch_size):
#     batch = df["Texte"].iloc[i:i+batch_size]
#     try:
#         r_batch, _ = do_predictions(get_prompt, batch, model="llama3.1:70b")
#         results.extend(r_batch)
#         indexes.extend(batch.index)

#         # Checkpoint every 100 samples
#         if i % 100 == 0:
#             df.loc[indexes, "prediction_simple"] = results
#             df.to_csv("checkpoint_predictions.csv", index=False)
#             print(f"Checkpoint sauvegardé à {i}.")

#         time.sleep(1)  # optional pause

#     except Exception as e:
#         print(f"Erreur au batch {i}: {e}")
#         continue

# df.loc[indexes, "prediction_simple"] = results
# df.to_csv("final_predictions.csv", index=False)


In [ ]:
# # Et même inspiration pour un truc encore plus complet, mais overkill pour l'instant

# import os
# import time
# import pandas as pd
# from datetime import datetime


# def ensure_dir(path):
#     if not os.path.exists(path):
#         os.makedirs(path)


# def timestamp():
#     return datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


# def do_predictions(prompt_func, texts, model):
#     prompt_name = prompt_func.__name__
#     results = [prompt_func(text) for text in texts]
#     return results, prompt_name


# def run_llm_batches(df, text_column, prompt_func, model,
#                     column_name=None, batch_size=20, sleep_time=1,
#                     checkpoint_every=100, output_dir="outputs"):

#     # Setup
#     start = time.time()
#     all_results = []
#     all_indexes = []
#     name = prompt_func.__name__ if column_name is None else column_name

#     ensure_dir(output_dir)

#     print(f"🚀 Début du traitement avec le prompt '{name}' et le modèle '{model}'")

#     # Boucle par batch
#     for i in range(0, len(df), batch_size):
#         batch = df[text_column].iloc[i:i + batch_size]
#         idx = batch.index

#         try:
#             batch_start = time.time()
#             r_batch, _ = do_predictions(prompt_func, batch, model=model)
#             all_results.extend(r_batch)
#             all_indexes.extend(idx)
#             print(f"✅ Batch {i}-{i + batch_size} traité en {time.time() - batch_start:.2f}s")

#             # Sauvegarde périodique
#             if (i // batch_size) % (checkpoint_every // batch_size) == 0:
#                 df.loc[all_indexes, name] = all_results
#                 ckpt_file = os.path.join(output_dir, f"checkpoint_{name}_{timestamp()}.csv")
#                 df.to_csv(ckpt_file, index=False)
#                 print(f"💾 Checkpoint enregistré : {ckpt_file}")

#             time.sleep(sleep_time)

#         except Exception as e:
#             print(f"⚠️ Erreur au batch {i}-{i + batch_size}: {e}")
#             continue

#     # Résultat final
#     df.loc[all_indexes, name] = all_results
#     final_file = os.path.join(output_dir, f"final_{name}_{timestamp()}.csv")
#     df.to_csv(final_file, index=False)

#     print(f"🏁 Traitement terminé en {time.time() - start:.2f}s")
#     print(f"📝 Résultat final enregistré : {final_file}")
#     return df
